<a href="https://colab.research.google.com/github/mharis0014/cv/blob/master/Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from zipfile import ZipFile
file_name = "dataset.zip"

with ZipFile (file_name, 'r') as zip:
  zip.extractall()
  print('Data extract from zip file')

Data extract from zip file


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os

In [3]:
num_classes = 5
img_rows, img_cols = 48, 48
batch_size = 32

training_data_dir = 'dataset/train'
validation_data_dir = 'dataset/validation'

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, shear_range=0.3, zoom_range=0.3,
                                   width_shift_range=0.4, height_shift_range=0.4, horizontal_flip=True,
                                   vertical_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)


In [5]:

train_generator = train_datagen.flow_from_directory(training_data_dir, color_mode='grayscale',
                                                    target_size=(img_rows,img_cols),
                                                    batch_size=batch_size, class_mode='categorical',
                                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(validation_data_dir, color_mode='grayscale',
                                                    target_size=(img_rows,img_cols),
                                                    batch_size=batch_size, class_mode='categorical',
                                                    shuffle=True)

Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [6]:
model = Sequential()

#Block 1
model.add(Conv2D(32,(3,3), padding='same', kernel_initializer='he_normal', 
          input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), padding='same', kernel_initializer='he_normal', 
          input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block 2
model.add(Conv2D(64,(3,3), padding='same', kernel_initializer='he_normal', 
          input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), padding='same', kernel_initializer='he_normal', 
          input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block 3
model.add(Conv2D(128,(3,3), padding='same', kernel_initializer='he_normal', 
          input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), padding='same', kernel_initializer='he_normal', 
          input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block 4
model.add(Conv2D(256,(3,3), padding='same', kernel_initializer='he_normal', 
          input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), padding='same', kernel_initializer='he_normal', 
          input_shape=(img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))


In [7]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [8]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('Emotion_detection.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=7,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

In [9]:
nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
755/755 [==============================] - ETA: 0s - loss: 1.8664 - accuracy: 0.2375
Epoch 00001: val_loss improved from inf to 1.53297, saving model to Emotion_detection.h5
755/755 [==============================] - 413s 547ms/step - loss: 1.8664 - accuracy: 0.2375 - val_loss: 1.5330 - val_accuracy: 0.3152
Epoch 2/25
755/755 [==============================] - ETA: 0s - loss: 1.5811 - accuracy: 0.2748
Epoch 00002: val_loss did not improve from 1.53297
755/755 [==============================] - 412s 546ms/step - loss: 1.5811 - accuracy: 0.2748 - val_loss: 1.5386 - val_accuracy: 0.3192
Epoch 3/25
755/755 [==============================] - ETA: 0s - loss: 1.5547 - accuracy: 0.2948
Epoch 00003: val_loss improved from 1.53297 to 1.51701, saving model to Emotion_detection.h5
755/755 [==============================] - 415s 549ms/step - loss: 1.5547 - accuracy: 0.2948 - val_loss: 1.5170 - val_accuracy: 0.327